In [1]:
import pandas as pd

import os
from glob import glob

In [2]:
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

dataset = 'ouro_volume_agricola_lavouras'

In [3]:
arquivos_csv = []
arquivos_csv.extend(glob(os.path.join(raiz, 'ouro_volume_agricola_lavouras*.csv')))
arquivos_csv.sort()

tabelas = (
    'Perm Culturas',
    'Perm Municípios',
    'Perm UF',
    'Temp Culturas',
    'Temp Municípios',
    'Temp UF',
    'Total-CF'
)

In [4]:
perm_munic = (arquivos_csv[0], arquivos_csv[1])
tab_perm_munic = (tabelas[0], tabelas[1])

temp_munic = (arquivos_csv[3], arquivos_csv[4])
tab_temp_munic = (tabelas[3], tabelas[4])

csv_uf = (arquivos_csv[2], arquivos_csv[5], arquivos_csv[6])
tab_uf = (tabelas[2], tabelas[5], tabelas[6])

In [24]:
with pd.ExcelWriter('/home/eduardo/Downloads/lavouras.xlsx', mode='w', engine="openpyxl") as export:
    for arquivo, tabela in zip(perm_munic, tab_perm_munic):
        df = (pd.read_csv(arquivo)
              .merge(pd.read_parquet(f'{raiz}/prata_ibge_geocodigo.parquet')
                     .filter(['UF-id', 'UF-sigla', 'municipio-id', 'municipio-nome']),
                     how='left',
                     left_on='geocodigo',
                     right_on='municipio-id'
                    )
        )

        if 'cultura' in df.columns:
              df = df.pivot_table(
                       index=['UF-id','UF-sigla', 'municipio-id', 'municipio-nome', 'cultura'],
                       columns='data',
                       values='area'
                       )
        else:
              df = df.pivot_table(
                       index=['UF-id','UF-sigla', 'municipio-id', 'municipio-nome'],
                       columns='data',
                       values='area'
                      )

        df.reset_index().to_excel(export, sheet_name=tabela, index=False)
    
    
    for arquivo, tabela in zip(temp_munic, tab_temp_munic):
        df = (pd.read_csv(arquivo)
              .merge(pd.read_parquet(f'{raiz}/prata_ibge_geocodigo.parquet')
                     .filter(['UF-id', 'UF-sigla', 'municipio-id', 'municipio-nome']),
                     how='left',
                     left_on='geocodigo',
                     right_on='municipio-id'
                    )
        )

        if 'cultura' in df.columns:
              df = df.pivot_table(
                      index=['UF-id','UF-sigla', 'municipio-id', 'municipio-nome', 'cultura', 'safra'],
                      columns='data',
                      values='valor'
                    )
        else:
              df = df.pivot_table(
                      index=['UF-id','UF-sigla', 'municipio-id', 'municipio-nome', 'safra'],
                      columns='data',
                      values='valor'
                    )

        df.reset_index().to_excel(export, sheet_name=tabela, index=False)

In [26]:
with pd.ExcelWriter('/home/eduardo/Downloads/lavouras.xlsx', mode='a', engine="openpyxl", if_sheet_exists='replace') as export:
    for arquivo, tabela in zip(csv_uf, tab_uf):
        df = (pd.read_csv(arquivo)
              .merge(pd.read_parquet(f'{raiz}/prata_ibge_geocodigo.parquet')
                     .filter(['UF-id', 'UF-sigla']),
                     how='left',
                     left_on='geocodigo',
                     right_on='UF-id'
                    )
        )

        if 'area' in df.columns:
              df = df.pivot_table(
                  index=['UF-id','UF-sigla'],
                  columns='data',
                  values='area'
                  )
        elif 'safra' in df.columns:
             df = df.pivot_table(
                  index=['UF-id','UF-sigla', 'safra'],
                  columns='data',
                  values='valor'
                  )
        else:
              df = df.pivot_table(
                  index=['UF-id','UF-sigla'],
                  columns='data',
                  values=['valor', 'cf']
                  )

        df.reset_index().to_excel(export, sheet_name=tabela)